In [1]:
from __future__ import division
import numpy as np
from glob import glob
from os.path import join, basename
from nipype.interfaces import afni

In [2]:
data_dir = '/scratch/PSB6351_2017/students/flannery/output/output_week7/'
mask_dir = '/scratch/PSB6351_2017/ds008_R2.0.0/preproc/anat_masks/'

smoothing_kernels = ['sm3', 'sm6', 'sm9', 'sm12']

f_acfs = np.empty((len(smoothing_kernels), 4))
for k, sm in enumerate(smoothing_kernels):
    sm_data_dir = join(data_dir, sm)
    
    subjects = sorted([basename(s) for s in glob(join(sm_data_dir, 'sub-*'))])
    s_acfs = np.empty((len(subjects), 4))
    for i, s in enumerate(subjects):
        mask_file = join(mask_dir, s, 'masks/_subject_id_{0}/wb_warped.nii.gz'.format(s))
        
        runs_dir = join(sm_data_dir, s, 'preproc/func/smoothed_highpassed/')
        run_files = sorted([basename(r) for r in glob(join(runs_dir, 'corr*run*.nii.gz'))])

        r_acfs = np.empty((len(run_files), 4))
        for j, r in enumerate(run_files):
            f = join(runs_dir, r)
            fwhm = afni.FWHMx(in_file=f, acf=True, mask=mask_file)
            res = fwhm.run()
            r_acfs[j, :] = np.array(res.outputs.acf_param)
            if sm=='sm12' and s=='sub-01' and j==0:
                dude = r_acfs[j, :]
        s_acf = np.mean(r_acfs, axis=0)
        s_acfs[i, :] = s_acf
    f_acf = np.mean(s_acfs, axis=0)
    f_acfs[k, :] = f_acf

170325-15:50:45,297 interface INFO:
	 stderr 2017-03-25T15:50:45.292280:++ 3dFWHMx: AFNI version=AFNI_16.3.18 (Dec 22 2016) [64-bit]
170325-15:50:45,803 interface INFO:
	 stderr 2017-03-25T15:50:45.803060:++ Authored by: The Bob
170325-15:50:45,805 interface INFO:
	 stderr 2017-03-25T15:50:45.803060:++ Number of voxels in mask = 26814
170325-15:50:47,335 interface INFO:
	 stderr 2017-03-25T15:50:47.335252:++ start FWHM calculations
170325-15:50:47,340 interface INFO:
	 stderr 2017-03-25T15:50:47.335252:** ERROR: (FAILED) attempt to over-write file corr_sub-01_task-conditionalstopsignal_run-01_bold_bet_afni_mask_bp_subbricks.out
170325-15:50:47,345 interface INFO:
	 stderr 2017-03-25T15:50:47.335252: + FWHM done (0.00 CPU s thus far)
170325-15:50:47,350 interface INFO:
	 stderr 2017-03-25T15:50:47.335252:++ start ACF calculations out to radius = 10.71 mm
170325-15:50:47,857 interface INFO:
	 stderr 2017-03-25T15:50:47.857591: + ACF done (0.00 CPU s thus far)
170325-15:50:47,858 interfac

In [3]:
print f_acfs
print np.mean(f_acf[:-1])

[[  0.85401361   2.21746831  10.1056134    4.02482614]
 [  0.61621029   3.13289962   9.11733052   8.34997157]
 [  0.4354184    4.97547781  11.0226794   12.94166905]
 [  0.43399844   6.96919148  13.424075    17.18273571]]
6.9424216381


In [4]:
np.array(res.outputs.acf_param)

array([  0.467974,   7.43604 ,  13.5702  ,  17.9675  ])